**Robotic Mapping with LiDAR Data |** Linear Transformations
============================================================



ROB 101: Computational Linear Algebra | University of Michigan, Department of Robotics





---



**Please read carefully. Ask questions if you are unsure.**
-----------------------------------------------------------



We use an auto-grader to check your work. If you invent new notation (such as, new variable names) for yourself, you will mess up the auto-grader and receive no points. We will NOT do manual regrades because of failure to use the requested variable names.



Do not reinitialize variables and data provided for you. Please just run the cells when information is initialized for you. DO NOT RETYPE IT unless it is in a static cell (a cell that has no run button).



Not all tests are visible to you. Just because you have passed a test, doesn’t mean you will get full credit. Take some time to understand what it is your code is doing and what should output so you can check your answers before submission.



### IllumiDesk Tips



* Within code cells, click the blue play button on the left to run the code.
* In the upper right, you can see if your kernel is connected. That is a dropdown menu that has additional functionality to “disconnect”, “restart”, and “clear all outputs” of the Kernel. None of these buttons will delete your work.
* Do not forget that code runs sequentially!
* Selecting the clipboard icon on the right will allow you to skip to different parts of this document without scrolling.
* Upon completion of an assignment, click the blue “Submit” button in the upper right.




---



In [ ]:
#Run this cell so you can use the necessary package(s).
using LinearAlgebra, Random, Plots, DelimitedFiles

default(fmt = :png)

for (root, dirs, files) in walkdir("/home/jovyan/")
  println("Root: ", root)
  println("Directories: ", dirs)
  println("Files: ", files)
end



---



Task 3
======



Now that you have seen the power of matrix transformations in 2D, let's move on to the more challenging case of 3D.



In this task, you will plot the raw point cloud data obtained from Cassie's LiDAR over a period of 1 second. Running Cassie's LiDAR at 10 Hz, we have collected 10 different scans of LiDAR data over the one second interval and combined them into one big file. You will see how huge the file is while working on this problem and appreciate what it means to do math at scale!



The collected data is in the frame of the LiDAR, which moves with the robot. To simplify the data management for you (because data management is not a focus of ROB 101), we have built a data parsing function that organizes the information from a slew of comma separated values (csv) files into different meaningful arrays. The function `data_parser()` will parse each second of your data set into five different arrays that you will need for this problem.



The input to the `data_parser()` is the **time interval id number** of the data set, with 𝑖𝑑∈\{9,10,11,12,13\} which gives the identity of a data set (*the corresponding time*). It must be passed as an argument to the function and **the data returned should be saved in the same order that it is produced by the parser**. See the second cell for an example function call.



The different arrays that will contain the data are given as:



* `pointcloud_data`: This has the xyz coordinates of the point cloud
* `Intensity_data`: The LiDAR Intensity data
* `R`: Rotation Matrix
* `t`: Translation vector
* `pose`: Gives the true xyz position (column 1) and orientation (column 2) of the robot


In [ ]:
# This is the data parser and helper functions block. 

function data_parser(id)
    # load point cloud from file
    
    #points = readdlm(string("data/Frame_", string(id),"/pointcloud_", string(id),".csv"),',');
    points = readdlm(string("pointcloud",string(id),".csv"),',');
    pointcloud_data = points[1:3,:]; # xyz
    Intensity_data = points[4,:]; # intensity
    
    # load the different transformation from the file, connect 
    # with the information learned in Task 1 
   
  #Transformations = readdlm(string("data/Frame_", string(id),"/Transformations_", string(id),".csv"),',');
  Transformations = readdlm(string("Transformations_",string(id),".csv"),',');
    R = Transformations[:,1:3]; # rotation
    t = Transformations[:,4]; # translation
    
    # Also get the position of the the robot from another file 
    
  #pose = readdlm(string("data/Frame_",string(id),"/Position_",string(id),".csv"),',');
    pose = readdlm(string("Position_",string(id),".csv"),',');
    return (pointcloud_data, Intensity_data, R, t, pose)
end


#= 
The next function will apply a homogeneous transformation to a matrix of points

This is the exact same process you were using in Tasks 1 and 2,
where we appended ones to the data, apply the homogeneous transformation, 
and then remove the ones after applying the transformation. 
This is how one organizes code to work at scale: you build functions! 
=#

function affine_transform(points, H)
    # Make a copy of the original points and append 1 to each point, 
    # (Homogenizing)
    transformed_p = [copy(points); ones(1,size(points,2))];
    
    # Transform 3D points, looping over all points and multiplying 
    # the Transformation matrix, exactly like Task 1!
    transformed_p = H * transformed_p
    
    # Remember to return the de-homogenized array, i.e.,
    # transformed_points should have same size as input points     
    return transformed_p[1:3,:];
end

#= 
A function to get robot heading at its position, this is more 
of a hack that we use to get the arrow of our position estimate when plotting, 
you are definitely not expected to understand it. 
=#

function robot_arrow(pose)
    position = pose[:,1]; # position
    (roll,pitch,yaw) = pose[:,2]; # orientation
    
    # extract rotation axis! 
    u = [1*cos(yaw-pi/2)]
    v = [1*sin(yaw-pi/2)]
    return (position, u, v)
end

#= 
Define the viewing window size 
for setting x and y axis limits, 
and other plotting parameters
=#
window_size = 15
min_x = -window_size
min_y = -window_size
max_y = window_size
min_z = 0
max_z = 2*window_size
max_x = window_size;
limits = ((min_x, max_x), (min_y, max_y), (min_z, max_z));


#=
    This function generates a 3D Point Cloud Plot
    Mandatory parameters are
                        xdata -> x coordinates
                        ydata -> y coordinates
                        zdata -> z coordinates
                        limits -> tuple of coordinate limits
                        color -> pointcloud color
=#
function plot_3D_pointcloud(xdata::Vector{Float64}, ydata::Vector{Float64}, zdata::Vector{Float64}, 
           limits, color::Vector{Float64};
            seriestype=:scatter3d, markersize=0.1, camera=(0,90), dpi=1080,
            markercolor=cgrad(:rainbow, rev=true), background_color=:black, 
            foreground_color=:black, legend=false, aspect_ratio=0.75, new=false)
    fxn = new ? plot : plot!
    fxn(xdata, ydata, zdata, xlims=limits[1], ylims=limits[2], zlims=limits[3], 
    seriestype=seriestype, markersize=markersize, camera=camera, dpi=dpi,
    markercolor=markercolor, marker_z=color, background_color=background_color,
    foreground_color=foreground_color, legend=legend, aspect_ratio=aspect_ratio,
    markerstrokewidth=0.001) 
end
#=
    This function plots the robot in 2D as a quiver
    Mandatory parameters are
                        xpos -> x coordinate of robot
                        ypos -> y coordinate of robot 
                        limits -> tuple of coordinate limits
                        gradient -> robot orientation (u,v)
=#
function plot_2D_robot(xpos::Vector{Float64}, ypos::Vector{Float64}, limits, gradient;
        seriestype=:quiver, projection="3d", arrow=(10,10), camera=(0,90), dpi=1080,
        background_color=:black, foreground_color=:black, legend=false, aspect_ratio=1, 
        seriescolor=:white, new=false)
    fxn = new ? plot : plot!
    fxn(xpos, ypos, seriescolor=seriescolor, seriestype=seriestype, projection=projection, 
        gradient=gradient, arrow=arrow, xlims=limits[1], ylims=limits[2], zlims=limits[3], 
        camera=camera, dpi=dpi, background_color=background_color, 
        foreground_color=foreground_color, legend=legend, aspect_ratio=aspect_ratio)
end


#=
    This function generates a 3D Point Cloud Plot
    Mandatory parameters are
                        xpos -> x coordinate of robot
                        ypos -> y coordinate of robot
                        zpos -> z coordinate of robot
=#
function plot_3D_robot(xpos::Vector{Float64}, ypos::Vector{Float64}, zpos::Vector{Float64}; 
                    markersize=4, seriestype=:scatter3d, seriescolor=:white, new=false)
    fxn = new ? plot : plot!
    fxn(xpos, ypos, zpos, seriescolor=seriescolor, markersize=markersize, seriestype=seriestype)
end

The next cell reads the raw point cloud in the last data collection instant, which is at time 13 seconds (`id = 13`). As discussed before, the function gives us point cloud data, intensity data, the rotation matrix `R`, the translation vector `t`, and finally an estimate of the robot's pose, that is, its position and orientation.



In [ ]:
#= 
Example call to the function to get the data 
at time  13 seconds of data collection
=#

# set the id number
id = 13

# parse the data and save it into different arrays
(point_data, intensity_data, R, t, pose) = data_parser(id)

# Lets see the size of the pointcloud and appreciate the challenge 
# of the problem at hand

# Can you say, six hundred ninety six thousand and four hundred ninety six columns? And 
# that is just for one second of data collected by Cassie. Now you see
# why Robotics needs math done at scale! 
size(point_data)

Task 3, Part A
--------------



As mentioned earlier, the LiDAR data is collected in the frame of the LiDAR while the robot is moving. Hence, each frame of LiDAR data is taken from a different position and orientation of the robot. This is analogous to you walking and simultaneously taking photos (not video) and then you trying to overlay the photos to create a single image. What a mess!



To underline the importance of keeping track of the position and orientation of Cassie while it is moving, we are going to overlay multiple LiDAR scans **without transforming** them to a common coordinate system (which we call a world frame). We are deliberately going to make a mess of things. You will see multiple images of something, like you are seeing double or triple!



In [ ]:
# We skip half of the data to shorten the time
start_index = 9
end_index  = 13

plot()
for id = start_index:2:end_index 
    # save the data into different arrays
    (point_data, intensity_data, R, t, pose) = data_parser(id)
    plot_3D_pointcloud(point_data[1,:], point_data[2,:], point_data[3,:], limits, intensity_data) 
end

#=
This plot could take 2 minutes or more to output
Don't wait any longer than 7 minutes.
=#
plot!()

We have multiple LiDAR images of something, but what that something might be is hard to say. To unravel this mystery, we're going to step back and process just one packet of LiDAR data before we engage with all of them again.



In [ ]:
 #=
This cell plots the robot at time 13 in the world frame
=#
(point_data, intensity_data, R, t, pose) = data_parser(end_index) 

# Build an arrow to indicate the robots position and orientation
(pos, u, v) = robot_arrow(pose);

# Plot the point cloud and white arrow representing the robot's pose
plot()  
plot_2D_robot(pos[1,:], pos[2,:], limits, (u,v))

The white arrow indicates the position and orientation of the robot. Now that we know the pose of Cassie, let’s also plot the raw point cloud in the same plot and see if the rings of the point cloud are consistent with Cassie's pose (i.e., position and orientation).



In [ ]:
(point_data, intensity_data, R, t, pose) = data_parser(end_index)

# Plot the 3D point cloud
# be patient with the time on this too! 
# shouldn't take longer than 3 mins
plot_3D_pointcloud(point_data[1,:], point_data[2,:], point_data[3,:], limits, intensity_data) 
plot!()

Take a close look at the plot you have just generated. **The LiDAR is mounted on top of the robot and the concentric circles you see in the LiDAR points should be centered about the robot.** What we see, instead, is that the data is translated and rotated with respect to the robot. We need to fix that by applying a homogeneous transformation to the point cloud data.



**You will help achieve this by building the affine transformation matrix,** `H`**:**



#### ****H = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix}****



**where…**



* `R` **is the given **3 × 3** rotation matrix**
* `t` **is the given **3 × 1** translation vector**
* ****0** is a **1 × 3** row vector of zeros**
* ****1** is just the number one**


💡Remember:



* Name your matrix `H`
* `R` and `t` have already been created for you by the data parser in the above cell!


In [ ]:
H = [R t;zeros(1,3) 1]is_it_correct_check1 = isapprox(H[:,1],[0.223755642141881,-0.968392974489662,-0.110220050665654,0.0], atol = 1e-3) ? "Yes" : "No"
is_it_correct_check2 = isapprox(H[:,4],[2.52669466213533,-0.957786861851155,1.4354495810566,1.0], atol = 1e-3) ? "Yes" : "No"


@show is_it_correct_check1; 
@show is_it_correct_check2;is_it_correct_check1 = isapprox(H[:,1],[0.223755642141881,-0.968392974489662,-0.110220050665654,0.0], atol = 1e-3) ? "Yes" : "No"
is_it_correct_check2 = isapprox(H[:,4],[2.52669466213533,-0.957786861851155,1.4354495810566,1.0], atol = 1e-3) ? "Yes" : "No"


@show is_it_correct_check1; 
@show is_it_correct_check2;is_it_correct_check1 = isapprox(H[:,1],[0.223755642141881,-0.968392974489662,-0.110220050665654,0.0], atol = 1e-3) ? "Yes" : "No"
is_it_correct_check2 = isapprox(H[:,4],[2.52669466213533,-0.957786861851155,1.4354495810566,1.0], atol = 1e-3) ? "Yes" : "No"


@show is_it_correct_check1; 
@show is_it_correct_check2;is_it_correct_check1 = isapprox(H[:,1],[0.223755642141881,-0.968392974489662,-0.110220050665654,0.0], atol = 1e-3) ? "Yes" : "No"
is_it_correct_check2 = isapprox(H[:,4],[2.52669466213533,-0.957786861851155,1.4354495810566,1.0], atol = 1e-3) ? "Yes" : "No"


@show is_it_correct_check1; 
@show is_it_correct_check2;

In the cell below, we are using your transformation matrix, `H`, to transform the point cloud data.



In [ ]:
#= 
affine transformation function performs the same actions you did in Task 1
It multiplies the transformation matrix, H, by the homogenized original data
Remember that homogenized just means a 1 has been appended to each point
=#
transformed_points = affine_transform(point_data, H)

println("Can you now see that the LiDAR rings are more centered about the robot?")
println("It should now look like the robot is facing the data it just collected.")
println("Previously, it may have looked like the robot took some data and then moved.")

# Visualize in 3D point cloud
plot() 
plot_3D_pointcloud(transformed_points[1,:], transformed_points[2,:], 
                    transformed_points[3,:], limits, intensity_data)

# Add robot pose to the plot 
plot_2D_robot(pos[1,:], pos[2,:], limits, (u,v))
plot!()

Task 3, Part B
--------------



#### **Building a map and gif and fusing different LiDAR scans!**



We now understand how to go from a sensor frame to a world frame. We will build a map by fusing data over multiple time periods. As you will notice from the figure you just made for one second of data, LiDAR scans collected over 1 second are not enough to give us full understanding of the scene and we therefore need to fuse multiple scans together for a more vivid picture. As much as we would like to give you the full data set (3 minutes of data), due to time constraints, we are going to have you work with five seconds (time = 9 to 13 seconds) of data.



However, **one takeaway from this problem should be that the robot is continuously moving, and we need to compensate for its motion with an appropriate transformation matrix that converts the LiDAR data in the sensor frame to LiDAR data in the world frame. The transformation has to be continuously estimated in real-time on the robot.** That part of the problem you can learn about in ROB 330, ROB 501, and ROB 530. There is an entire research area that focuses on calculating the best approximate transformations from the moving LiDAR to the fixed world frame.



**Your job is to add code to the for loop that allows us to iterate over all point clouds over the time intervals from 9 to 13.**



In [ ]:
start_index = 9
end_index  = 13
Plots.plot();

In [ ]:
#=
Code Skeleton : Build an animation!
Be patient with the gif! 
You may need to close some tabs and free
up your local RAM.
=#

anim = @animate for id = start\_index:end\_index 
 transformed\_points = [];

 
 #= 
 YOUR CODE HERE (3 lines described below)
 
 Line 1. Parse the data for the i-th time interval (id) using the 
 data\_parser() function. Store it in the arrays with the same name 
 like we did before in the example before Part A.

 Line 2. Build the affine transformation matrix, H, from Part A

 Line 3. Transform your point cloud using the affine transformation function
 and store the result in a variable named transformed\_points as 
 exemplified in Part A
 =#
 
 plot\_3D\_pointcloud(transformed\_points[1,:], transformed\_points[2,:], 
 transformed\_points[3,:],limits, intensity\_data)
 
 # A Hack for plotting the position
 if(id!=start\_index)
 # Plot your previous position with a black dot everytime except the first
 (point\_data, intensity\_data, R, t, prev\_pose) = data\_parser(id-1); 
 plot\_3D\_robot([prev\_pose[1,1]],[prev\_pose[2,1]],
 [prev\_pose[3,1]], seriescolor=:black)
 end
 
 plot\_3D\_robot([pose[1,1]],[pose[2,1]],[pose[3,1]],seriescolor=:white)

 prev\_pose = copy(pose);
end

# replace "yourName" with your actual first and last name (no spaces)
gif(anim, "yourName\_RobotGIF.gif", fps = 1)

In [ ]:
#=
Code Skeleton : Build an animation!
Be patient with the gif! 
You may need to close some tabs and free
up your local RAM.
=#

anim = @animate for id = start_index:end_index 
    transformed_points = [];

    (point_data, intensity_data, R, t, pose) = data_parser(id)
    H = [R t;zeros(1,3) 1]
    transformed_points = affine_transform(point_data, H)
    #= 
      YOUR CODE HERE (3 lines described below)
      
      Line 1. Parse the data for the i-th time interval (id) using the 
        data_parser() function. Store it in the arrays with the same name 
        like we did before in the example before Part A.

      Line 2. Build the affine transformation matrix, H, from Part A

      Line 3. Transform your point cloud using the affine transformation function
        and store the result in a variable named transformed_points as 
        exemplified in Part A
    =#
    
    plot_3D_pointcloud(transformed_points[1,:], transformed_points[2,:], 
            transformed_points[3,:],limits, intensity_data)
    
    # A Hack for plotting the position
    if(id!=start_index)
        # Plot your previous position with a black dot everytime except the first
        (point_data, intensity_data, R, t, prev_pose) = data_parser(id-1); 
        plot_3D_robot([prev_pose[1,1]],[prev_pose[2,1]],
              [prev_pose[3,1]], seriescolor=:black)
    end
     
    plot_3D_robot([pose[1,1]],[pose[2,1]],[pose[3,1]],seriescolor=:white)

    prev_pose = copy(pose);
end

# replace "yourName" with your actual first and last name (no spaces)
gif(anim, "kaichiweng_RobotGIF.gif", fps = 1)

You should see the white dot, Cassie, moving within the map while everything else in the map stays fixed (*like the stairs*)!





---



🎉 Congrats! You completed your first ROB101 project and applied your knowledge to real world data! ✨👏🏽
------------------------------------------------------------------------------------------------------



![](https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExMDNycHF3eWszcjhwZHM1OXZ1NTdqd3BtdmVvZjZydHQ2c293YmowZiZlcD12MV9naWZzX3NlYXJjaCZjdD1n/3gpfkdQIunv5C/200.gif)